<a href="https://colab.research.google.com/github/haoboooo/analyzing-temporal-effect-of-sentiment-analysis/blob/main/process_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re
import gzip
import json

## read reviews

In [2]:
file1 = "./drive/MyDrive/Colab Notebooks/CL/Software.json.gz"
file2 = "./drive/MyDrive/Colab Notebooks/CL/AMAZON_FASHION.json.gz"
file3 = "./drive/MyDrive/Colab Notebooks/CL/Video_Games.json.gz"

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [4]:
data1 = getDF(file1)

In [5]:
data2 = getDF(file2)

In [6]:
data3 = getDF(file3)

## split data

In [7]:
data1.head(3)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7,NaN


In [8]:
data2.head(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2.0,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3,NaN,NaN
2,4.0,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN


In [9]:
data3.head(2)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3.0,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN


In [10]:
def extract(df):
  dup = df.duplicated(subset='reviewerID').sum() # Check duplicate values for a column
  if dup != 0:
    df=df.drop_duplicates(subset=['reviewerID']) # drop duplicate values for a column

  df = df[["overall","reviewTime","reviewText"]]
  df["overall"] = df["overall"].replace(5.0,1)
  df["overall"] = df["overall"].replace(4.0,1)
  df["overall"] = df["overall"].replace(1.0,-1)
  df["overall"] = df["overall"].replace(2.0,-1)
  df["overall"] = df["overall"].replace(3.0,0)
  df["reviewTime"] =  df["reviewTime"].apply(lambda x: x[-4:])
  df = df.dropna(axis=0,how='any')
  years = list(df["reviewTime"].unique())
  shape = df.shape
  return df,shape,years

In [11]:
software,shape1,years1 = extract(data1)
print(shape1)
print(years1)
software.groupby('reviewTime').count()

(375086, 3)
['2014', '2013', '2011', '2010', '2009', '2012', '2008', '2007', '2018', '2017', '2016', '2015', '2006', '2005', '2001', '2003', '2002', '2000', '2004', '1999', '1998']


,overall,reviewText
reviewTime,,
1998,1,1
1999,120,120
2000,1113,1113
2001,2356,2356
2002,3257,3257
2003,3784,3784
2004,3696,3696
2005,5531,5531
2006,7173,7173


In [12]:
fashion,shape2,years2 = extract(data2)
print(shape2)
print(years2)
fashion.groupby('reviewTime').count()

(748158, 3)
['2014', '2013', '2012', '2010', '2017', '2016', '2015', '2011', '2007', '2006', '2004', '2008', '2005', '2003', '2009', '2018', '2002']


,overall,reviewText
reviewTime,,
2002,5,5
2003,19,19
2004,51,51
2005,144,144
2006,311,311
2007,1375,1375
2008,1599,1599
2009,1839,1839
2010,1977,1977


In [17]:
game,shape3,years3 = extract(data3)
print(shape3)
print(years3)
game.groupby('reviewTime').count()

(1539344, 3)
['2014', '2013', '2012', '2010', '2017', '2016', '2015', '2011', '2006', '2009', '2018', '2007', '2008', '2005', '2004', '2001', '2000', '2003', '2002', '1999', '1998', '1997']


,overall,reviewText
reviewTime,,
1997,5,5
1998,4,4
1999,728,728
2000,6645,6645
2001,14712,14712
2002,16793,16793
2003,14831,14831
2004,14126,14126
2005,13808,13808


In [14]:
sw1 = software[(software["reviewTime"] == "1998")|(software["reviewTime"] == "1999")|(software["reviewTime"] == "2000")|(software["reviewTime"] == "2001")|(software["reviewTime"] == "2002")|(software["reviewTime"] == "2003")|(software["reviewTime"] == "2004")|(software["reviewTime"] == "2005")|(software["reviewTime"] == "2006")]
sw2 = software[(software["reviewTime"] == "2007")|(software["reviewTime"] == "2008")|(software["reviewTime"] == "2009")|(software["reviewTime"] == "2010")|(software["reviewTime"] == "2011")|(software["reviewTime"] == "2012")]
sw3 = software[(software["reviewTime"] == "2013")|(software["reviewTime"] == "2014")|(software["reviewTime"] == "2015")|(software["reviewTime"] == "2016")|(software["reviewTime"] == "2017")]
sw4 = software[(software["reviewTime"] == "2018")]

print(sw1.shape,sw2.shape,sw3.shape,sw4.shape)

(27031, 3) (83780, 3) (253161, 3) (11114, 3)


In [15]:
fa1 = fashion[(fashion["reviewTime"] == "2002")|(fashion["reviewTime"] == "2003")|(fashion["reviewTime"] == "2004")|(fashion["reviewTime"] == "2005")|(fashion["reviewTime"] == "2006")]
fa2 = fashion[(fashion["reviewTime"] == "2007")|(fashion["reviewTime"] == "2008")|(fashion["reviewTime"] == "2009")|(fashion["reviewTime"] == "2010")|(fashion["reviewTime"] == "2011")|(fashion["reviewTime"] == "2012")]
fa3 = fashion[(fashion["reviewTime"] == "2013")|(fashion["reviewTime"] == "2014")|(fashion["reviewTime"] == "2015")|(fashion["reviewTime"] == "2016")|(fashion["reviewTime"] == "2017")]
fa4 = fashion[(fashion["reviewTime"] == "2018")]

print(fa1.shape,fa2.shape,fa3.shape,fa4.shape)

(530, 3) (16800, 3) (680451, 3) (50377, 3)


In [27]:
gm1 = game[(game["reviewTime"]=="1997")|(game["reviewTime"]=="1998")|(game["reviewTime"]=="1999")|(game["reviewTime"]=="2000")|(game["reviewTime"]=="2001")|(game["reviewTime"]=="2002")|(game["reviewTime"]=="2003")|(game["reviewTime"]=="2004")|(game["reviewTime"]=="2005")|(game["reviewTime"]=="2006")]
gm2 = game[(game["reviewTime"] == "2007")|(game["reviewTime"] == "2008")|(game["reviewTime"] == "2009")|(game["reviewTime"] == "2010")|(game["reviewTime"] == "2011")|(game["reviewTime"] == "2012")]
gm3 = game[(game["reviewTime"] == "2013")|(game["reviewTime"] == "2014")|(game["reviewTime"] == "2015")|(game["reviewTime"] == "2016")|(game["reviewTime"] == "2017")]
gm4 = game[(game["reviewTime"] == "2018")]

print(gm1.shape,gm2.shape,gm3.shape,gm4.shape)

(95400, 3) (273545, 3) (1082462, 3) (87937, 3)


## store the data

In [18]:
dir1 = "./drive/MyDrive/Colab Notebooks/CL/software/"
dir2 = "./drive/MyDrive/Colab Notebooks/CL/fashion/"
dir3 = "./drive/MyDrive/Colab Notebooks/CL/game/"

In [19]:
sw1.to_csv(dir1+"98_06.csv")
sw2.to_csv(dir1+"07_12.csv")
sw3.to_csv(dir1+"13_17.csv")
sw4.to_csv(dir1+"18.csv")

In [20]:
fa1.to_csv(dir2+"02_06.csv")
fa2.to_csv(dir2+"07_12.csv")
fa3.to_csv(dir2+"13_17.csv")
fa4.to_csv(dir2+"18.csv")

In [28]:
gm1.to_csv(dir3+"97_06.csv")
gm2.to_csv(dir3+"07_12.csv")
gm3.to_csv(dir3+"13_17.csv")
gm4.to_csv(dir3+"18.csv")